In [17]:
import numpy as np
import pandas as pd
import torch
import pickle
import torch.nn as nn
import glob
import transformers
from bs4 import BeautifulSoup
import requests
import re
import time
from collections import defaultdict
import pdfplumber
from tqdm import tqdm
import collections
from selenium import webdriver
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset

import sys

sys.path.insert(0, '../src/models/')
import predict_model

In [18]:
# Load BERT
model = predict_model.loadBERT("../models/", 'model_weights_splitted_reducednegatives.pt')
# Load the BERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU Success


In [19]:
def SpanPredictor(span, pred_values=False, threshold=False):
      
    """
    Uses a trained bert classifier to see if a span
    belongs to a species description or otherwise.
    """
    
    # Clean text
    TextCleaner = [
        '\(\d+.+?Close\n\t\n\)',
        '\[\d*\]',
        '\([^)]*\)',
        '<[^<]+>',
    ]
    
    for Cleaner in TextCleaner:
        span = re.sub(Cleaner, '', span, flags=re.DOTALL)
    
    with torch.no_grad():
        # Tokenize input
        inputs = tokenizer(span, return_tensors="pt", truncation=True)
        # Predict class
        outputs = model(inputs['input_ids'], inputs['attention_mask'])
        # Get prediction values
        exps = torch.exp(outputs)
        # Get class
        span_class = exps.argmax(1).item()

        # Print the prediction values
        if pred_values:
            return span_class, exps[0]
        else:
            return span_class    

In [36]:
ipni_links = pickle.load(open('../data/processed/intermediate_ipni_links_trees.pkl', 'rb'))

# Extract the indices
ipni_index = [link.strip('https://www.ipni.org/n/') for link in ipni_links]
# Create powo links
powo_links = ['http://powo.science.kew.org/taxon/urn:lsid:ipni.org:names:' + index for index in ipni_index]

EOFError: Ran out of input

In [28]:
URL = 'https://www.ipni.org/?perPage=500&page=1&q=species%3A*'

In [29]:
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [21]:
preds = []
for span in soup.find_all('dd'):
    span = span.text.strip()
    preds.append((span, SpanPredictor(span, pred_values=True)))

In [27]:
soup.title.text.split(' | ')[0]

'Ceiba pentandra (L.) Gaertn.'

In [30]:
refs = [ref for ref in soup.find_all('a')]